# Data available through the API is not complete!
However, we have a web link for each award in our json data. We can use to scrape the rest! <br>
Some missing values are marked N/A or unavailable

In [1]:
import requests
import os
import json
import glob
import pandas as pd
from bs4 import BeautifulSoup

In [3]:
one_file_dir = os.path.join(os.pardir,'data', 'raw', '1996.json')

json library is faster than using pandas

In [4]:
# # %%timeit
# df_year = pd.read_json(one_file_dir)
# award_links = df_year.link.tolist()

In [62]:
# %%timeit
with open(one_file_dir) as f:
    year = json.load(f)
award_links = [y.get('link') for y in year]

CPU times: user 11 µs, sys: 1e+03 ns, total: 12 µs
Wall time: 21.5 µs


# Experimenting

In [5]:
link = award_links[0]
link

'https://www.sbir.gov/sbirsearch/detail/814223'

In [6]:
# scrape more data
# r = requests.get(award_links[0])
r = requests.get(link)

In [7]:
r.status_code

200

In [19]:
r.headers

{'Date': 'Wed, 13 Jun 2018 12:28:51 GMT', 'Content-Type': 'text/html; charset=utf-8', 'Transfer-Encoding': 'chunked', 'Connection': 'keep-alive', 'Server': 'Apache', 'Strict-Transport-Security': 'max-age=31536000; includeSubDomains; preload', 'X-Content-Type-Options': 'nosniff, nosniff', 'X-Drupal-Cache': 'MISS', 'Expires': 'Sun, 19 Nov 1978 05:00:00 GMT', 'Cache-Control': 'public, max-age=600', 'Content-Language': 'en', 'X-Frame-Options': 'SAMEORIGIN', 'Etag': '"1528892931-1"', 'Last-Modified': 'Wed, 13 Jun 2018 12:28:51 GMT', 'Vary': 'Cookie,Accept-Encoding', 'Content-Encoding': 'gzip'}

In [20]:
input_soup = BeautifulSoup(r.content, 'html5lib')

In [21]:
input_soup

<!DOCTYPE html>
<html dir="ltr" lang="en" prefix="content: http://purl.org/rss/1.0/modules/content/ dc: http://purl.org/dc/terms/ foaf: http://xmlns.com/foaf/0.1/ og: http://ogp.me/ns# rdfs: http://www.w3.org/2000/01/rdf-schema# sioc: http://rdfs.org/sioc/ns# sioct: http://rdfs.org/sioc/types# skos: http://www.w3.org/2004/02/skos/core# xsd: http://www.w3.org/2001/XMLSchema#"><head>
  <link href="http://www.w3.org/1999/xhtml/vocab" rel="profile"/>
  <meta content="width=device-width, initial-scale=1.0" name="viewport"/>
  <!--[if IE]><![endif]-->
<meta content="IE=edge" http-equiv="X-UA-Compatible"/>
<meta content="text/html; charset=utf-8" http-equiv="Content-Type"/>
<link href="https://www.sbir.gov/sites/all/themes/sbir/favicon.png" rel="shortcut icon" type="image/png"/>
<meta content="always" name="referrer"/>
  <title>Novel Dopant Strategies for the Development of Submicron PZT Ceramics | SBIR.gov</title>
  <link href="/sites/default/files/advagg_css/css__1X0zSbp3a6upgwmFDVMw3bKkE6n

In [22]:
# list of information
info_list = []

In [23]:
title = input_soup.title.text.split('|')[0]
info_list.append( ('title', title) )

In [24]:
# relevant information are below this node
imp_info = input_soup.find('div', class_= "award-info-wrapper")

In [34]:
type(imp_info), imp_info

(bs4.element.Tag, <div class="award-info-wrapper">
             <div class="info-wrapper open-section-title">Award Information</div>
       <div class="container-fluid">
         <div class="row open-style">
           <div class="col-md-6">
             <span class="open-label">Agency:</span> <span class="open-description">Department of Defense</span>
           </div>
           <div class="col-md-6">
             <span class="open-label">Branch:</span> <span class="open-description">Navy</span>
           </div>
         </div>
         <div class="row open-style">
           <div class="col-md-6">
             <span class="open-label">Contract:</span> <span class="open-description">N00014-96-C-0310</span>
           </div>
           <div class="col-md-6">
             <span class="open-label">Agency Tracking Number:</span> <span class="open-description">300300287</span>
           </div>
         </div>
         <div class="row open-style">
           <div class="col-md-3">
      

In [26]:
# Agency and Branch
top_info = imp_info.findAll('div', class_ = 'col-md-6', limit=5)

In [27]:
layer1 = [(i.find('span', class_='open-label').text.strip(':'),
          i.find('span', class_='open-description').text) for i in top_info]

In [28]:
info_list.extend(layer1)

In [29]:
# solicitation, award amount, ...
mid_info = imp_info.findAll('div', class_ = 'col-md-3', limit=9)

In [30]:
layer2 = [(i.find('span', class_='open-label').text.strip(':'),
          i.find('span', class_='open-description').text) for i in mid_info]

In [31]:
info_list.extend(layer2)

In [49]:
info_list.append( ('company name', imp_info.find('div', class_='sbc-name-wrapper').text.strip() ) )
# imp_info.find('div', class_='sbc-name-wrapper').text.strip()

'TRS CERAMICS, INC.'

In [33]:
info_list.append( ('company address', imp_info.find('div', class_= 'sbc-address-wrapper').text.strip() ) )

In [61]:
# DUNS
duns = imp_info.find('div', class_ = 'col-md-12')
info_list.append( (duns.find('span', class_='open-label').text.strip(':'),
          duns.find('span', class_='open-description').text) )

In [62]:
people_info = imp_info.find('div', class_='col-md-4')
pi = people_info.find('div', class_='award-sub')
pi_d = pi.find_all_next('span', class_='award-node-section-label', limit=4)
pi_info = dict([(i.text.strip(':'), i.next_sibling.strip('\xa0').replace('\xa0', ' ')) for i in pi_d])
info_list.append( (pi.text, pi_info) )

In [63]:
contact = people_info.find_next_sibling('div')
contact_sub = contact.find('div', class_='award-sub')

try:
    name_str = contact_sub.find_next('span', class_='award-node-section-label')
    contact_info = {}
    contact_info[name_str.text.strip(':')] = name_str.next_sibling.strip('\xa0').replace('\xa0', ' ')
    contact_rest = [ i.replace('\xa0', '').split(':')  for i in name_str.next_siblings if isinstance(i,str) and ':' in i]
    contact_info.update(dict(contact_rest))
except:
    name_str = contact_sub.find_next('div', class_='award-sub-description')
    contact_rest = [ i.replace('\xa0', '').strip().split(':')  for i in name_str.contents if isinstance(i,str) and ':' in i]
    contact_info = dict(contact_rest)

info_list.append( (contact_sub.text, contact_info) )

In [64]:
inst = contact.find_next_sibling('div')
inst_key = inst.find('div', class_='award-sub')
ri = inst.select_one('div[class=award-sub-description] > div').text.strip()
# if not ri == 'N/A':
info_list.append( (inst_key.text, ri) )

In [65]:
# abstract = imp_info.find('div', class_='abstract-wrapper')
# key = abstract.find('div', class_='info-wrapper')
key = imp_info.select_one('div[class=abstract-wrapper] > div[class=info-wrapper]')
abs_text = key.next_sibling.strip()
# if not abs_text == 'N/A':
info_list.append( (key.text, abs_text) )

In [66]:
dict(info_list)

{'title': 'Iconic, Graphic, Data Flow Programming for High-Performance Real-Time Workstations ',
 'Agency': 'Department of Defense',
 'Branch': 'Navy',
 'Contract': 'N/A',
 'Agency Tracking Number': '29022',
 'Socially and Economically Disadvantaged': 'N',
 'Amount': '$150,000.00',
 'Phase': 'Phase II',
 'Program': 'SBIR',
 'Awards Year': '1996',
 'Solicitation Year': 'N/A',
 'Solicitation Topic Code': 'N/A',
 'Solicitation Number': 'N/A',
 'HUBZone Owned': 'N',
 'Woman Owned': 'N',
 'company name': 'Management Communications & Control, I',
 'company address': '2000 North 14th Street Suite, 220, Arlington, VA, 22201',
 'DUNS': 'N/A',
 'Principal Investigator': {'Name': 'Christopher B. Robbins',
  'Phone': '(703) 522-7177'},
 'Business Contact': {'Phone': '() -'},
 'Abstract': 'Management Communications and Control, Inc. (MCCI) proposes development of a PGM environment for desktop signal processing workstations. A domain engineering inter- face will provide an open architecture to all s

WHAT I NEED TO SCRAPE:
Agency
Branch
Amount
Company address
PI Name and title
Business Contact Name and title
DUNS (Small Business Identification Number)
Solicitation Year
Solicitation Topic Code
Sollicitation number
Contract
Agency Tracking Number
HUBZone Owned
Woman Owned
Socially and Economically Disadvantage